In [1]:
# Setup
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
import spacy
from optimized_anchor import anchor_text, anchor_base
import pickle
import myUtils
from myUtils import *
from dataset.dataset_loader import *
import datetime
%load_ext line_profiler

SEED = 84
torch.manual_seed(SEED)
warnings.simplefilter("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# can be sentiment/spam/offensive
dataset_name = 'sentiment'
text_parser, label_parser, ds_train, ds_val = get_dataset(dataset_name)

Number of tokens in training samples: 6407
Number of tokens in training labels: 2


In [3]:
train, train_labels, test, test_labels, anchor_examples = preprocess_examples(ds_train)

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
ignored = get_ignored(anchor_examples)
normal_occurences = get_occurences(anchor_examples)
anchor_base.AnchorBaseBeam.best_group = BestGroup('check', normal_occurences, filter_anchors = False, desired_optimize = False)

## notice!

In [6]:
ignored = []

In [7]:
anchor_examples = anchor_examples[:2]

In [8]:
!nvidia-smi

Wed Aug 10 22:17:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.54       Driver Version: 510.54       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:AF:00.0 Off |                  Off |
| 33%   37C    P8    17W / 260W |      8MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
optimize = True
anchor_text.AnchorText.set_optimize(optimize)
explainer = anchor_text.AnchorText(nlp, ['positive', 'negative'], use_unk_distribution=False)

In [10]:
from transformers import AutoTokenizer
model = torch.jit.load(f'models/tinybert/{dataset_name}/traced.pt').to(device)
model = model.eval()
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', use_fast = False)
myUtils.model = model
myUtils.tokenizer = tokenizer

In [100]:
tokenized = explainer.tg.bert_tokenizer(["i [MASK] to [MASK] party yesterday"], 
          add_special_tokens=True, return_tensors="pt", padding=True)["input_ids"]

In [101]:
tokenized

tensor([[ 101, 1045,  103, 2000,  103, 2283, 7483,  102]])

In [105]:
(tokenized[0] == explainer.tg.bert_tokenizer.mask_token_id).nonzero().squeeze(1)

tensor([2, 4])

In [13]:
tokenized_seperate = [explainer.tg.bert_tokenizer.encode("i went to a party yesterday", 
          add_special_tokens=True) for _ in range(100)]
tokenized_seperate = [torch.tensor([t], device=device) for t in tokenized_seperate]

In [130]:
tokenizer.convert_ids_to_tokens([1,2])

['[unused0]', '[unused1]']

In [134]:
#torch._C._jit_set_texpr_fuser_enabled(False)
my_utils = TextUtils(anchor_examples, test, explainer, predict_sentences, ignored, f"profile.pickle", optimize=optimize)
myUtils.model = model
myUtils.tokenizer = tokenizer
set_seed()
%prun -s cumtime -T check/profile.txt my_utils.compute_explanations(list(range(len(anchor_examples))))

number 0
[0.18181818181818182]
[0.2727272727272727]
[0.2727272727272727]
[0.7142857142857143]
[0.971830985915493]
[0.8571428571428571]
[0.5454545454545454]
[0.18181818181818182]
[0.36363636363636365]
number 1
[0.36363636363636365]
[0.2727272727272727]
[0.2727272727272727]
[0.18181818181818182]
[0.2727272727272727]
[0.2727272727272727]
[0.36363636363636365]
[0.18181818181818182]
[0.09090909090909091]
[0.45454545454545453]
[0.5454545454545454]
[0.2727272727272727]
[0.09090909090909091]
[0.45454545454545453]
 
*** Profile printout saved to text file 'check/profile.txt'. 


         1355547 function calls (1355347 primitive calls) in 0.862 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.865    0.865 {built-in method builtins.exec}
        1    0.000    0.000    0.865    0.865 <string>:1(<module>)
        1    0.000    0.000    0.865    0.865 myUtils.py:290(compute_explanations)
        2    0.002    0.001    0.863    0.432 myUtils.py:265(get_exp)
        2    0.000    0.000    0.861    0.431 anchor_text.py:250(explain_instance)
        2    0.001    0.000    0.761    0.380 anchor_base.py:255(anchor_beam)
       63    0.004    0.000    0.749    0.012 anchor_text.py:209(sample_fn)
       59    0.000    0.000    0.730    0.012 anchor_base.py:217(<lambda>)
       59    0.002    0.000    0.730    0.012 anchor_base.py:186(complete_sample_fn)
       63    0.005    0.000    0.649    0.010 anchor_text.py:105(sample)
       86    0.002    0.000    0.616    0.007 anchor

In [15]:
###### my_utils = TextUtils(anchor_examples, test, explainer, predict_sentences, ignored,f"profile.pickle")
#%lprun -s -m modified_anchor.anchor_text -m modified_anchor.anchor_base -m myUtils -T profile.txt  my_utils.compute_explanations(list(range(len(anchor_examples))))

In [16]:
print(datetime.datetime.now())

2022-08-10 22:17:58.159443
